In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, SpatialDropout2D, ELU
from keras.layers import Convolution2D, MaxPooling2D, Cropping2D
from keras.layers.core import Lambda

from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Input, InputLayer, Lambda
from keras.layers import Convolution2D, MaxPooling2D, Flatten, SpatialDropout2D, Cropping2D
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing import image

from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import train_test_split
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
import os
import csv
import cv2
import numpy as np
import sklearn

samples = []
with open('my_data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

print("Samples: ", len(samples))

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)


Samples:  4033


In [3]:
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = 'my_data/IMG/'+batch_sample[0].split('/')[-1]
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            print("X_train: ", X_train)
            print("y_train: ", y_train)
            yield sklearn.utils.shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)


In [4]:
model = Sequential()
model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(66, 200, 3), name='Normalization'))  # Normalization courtesy of John Chen
model.add(Convolution2D(24, 5, 5, subsample=(2,2), activation='relu', name='Conv1'))
model.add(Convolution2D(36, 5, 5, subsample=(2,2), activation='relu', name='Conv2'))
model.add(Convolution2D(48, 5, 5, subsample=(2,2), activation='relu', name='Conv3'))
model.add(Convolution2D(64, 3, 3, activation='relu', name='Conv4'))
model.add(Convolution2D(64, 3, 3, activation='relu', name='Conv5'))
model.add(Flatten())
model.add(Dense(1164, activation='relu', name='FC1'))
model.add(Dense(100, activation='relu', name='FC2'))
model.add(Dense(50, activation='relu', name='FC3'))
model.add(Dense(10, activation='relu', name='FC4'))
model.add(Dense(1, name='output'))
model.summary()

C:\Users\Abhishek Jadav\Miniconda3\envs\carnd-term1\lib\site-packages\ipykernel\__main__.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), name="Conv1", activation="relu", strides=(2, 2))`
  app.launch_new_instance()
C:\Users\Abhishek Jadav\Miniconda3\envs\carnd-term1\lib\site-packages\ipykernel\__main__.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), name="Conv2", activation="relu", strides=(2, 2))`
C:\Users\Abhishek Jadav\Miniconda3\envs\carnd-term1\lib\site-packages\ipykernel\__main__.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), name="Conv3", activation="relu", strides=(2, 2))`
C:\Users\Abhishek Jadav\Miniconda3\envs\carnd-term1\lib\site-packages\ipykernel\__main__.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name="Conv4")`
C:\Users\Abhishek Jadav\Miniconda3\envs\carnd-term1\lib\site-packages\ipykernel\__main__.py:

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Normalization (Lambda)       (None, 66, 200, 3)        0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 31, 98, 24)        1824      
_________________________________________________________________
Conv2 (Conv2D)               (None, 14, 47, 36)        21636     
_________________________________________________________________
Conv3 (Conv2D)               (None, 5, 22, 48)         43248     
_________________________________________________________________
Conv4 (Conv2D)               (None, 3, 20, 64)         27712     
_________________________________________________________________
Conv5 (Conv2D)               (None, 1, 18, 64)         36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1152)              0         
__________

In [5]:
opt = Adam(lr=0.0001)
model.compile(optimizer=opt, loss='mse', metrics=[])





In [6]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        print('BEGIN TRAINING')
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [7]:
nb_epoch = 2
batch_history = LossHistory()
# 4 epochs of 28k
h = model.fit_generator(train_generator, 
                    samples_per_epoch=len(train_samples), 
                    validation_data=validation_generator,
                    nb_val_samples=len(validation_samples), nb_epoch=nb_epoch
                    verbose = 1, callbacks=[batch_history]) # low initial loss = bad

C:\Users\Abhishek Jadav\Miniconda3\envs\carnd-term1\lib\site-packages\ipykernel\__main__.py:7: UserWarning: The semantics of the Keras 2 argument  `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Update your method calls accordingly.
C:\Users\Abhishek Jadav\Miniconda3\envs\carnd-term1\lib\site-packages\ipykernel\__main__.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=2, steps_per_epoch=3226, validation_steps=807, validation_data=<generator...)`


Epoch 1/2
X_train:  [None None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None None
 None None]
y_train:  [ 0.          0.6500001  -0.5843893   0.          0.         -0.3463798   0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.         -0.35        0.          0.          0.          0.
 -0.05       -0.35        0.         -0.08409464  0.          0.          0.
  0.          0.          0.          0.        ]
X_train:  [None None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None None
 None None]
y_train:  [ 0.         0.         0.         0.         0.        -0.2       -0.5071558
  0.         0.6875544  0.         0.         0.        -0.2        0.         0.
  0.        -0.05       0.         0.         0.         0.         0.         0.
 -0.35       0.       

ValueError: Error when checking model input: expected Normalization_input to have 4 dimensions, but got array with shape (32, 1)